In [10]:
import os
from PIL import Image 
import torch
from torch.utils.data import Dataset , DataLoader
from torchvision import transforms, datasets, utils
import numpy as np
from tqdm import tqdm
import numpy as np
import torch 
import torch.nn as nn
import torchvision.transforms.functional as TF 
import torch.optim as optim
import torch.nn.functional as F

In [11]:
class CityscapesDataset(Dataset):
    def __init__(self, split, root_dir, mode='fine', transform=None, eval=False):
        self.transform = transform
        if mode == 'fine':
            self.mode = 'gtFine'
        self.split = split
        self.yLabel_list = []
        self.XImg_list = []
        self.eval = eval

        self.label_path = os.path.join(os.getcwd(), root_dir+'\\'+self.mode+'\\'+self.split)
        self.rgb_path = os.path.join(os.getcwd(), root_dir+'\\leftImg8bit\\'+self.split)
        city_list = os.listdir(self.label_path)
        for city in city_list:
            temp = os.listdir(self.label_path+'/'+city)
            list_items = temp.copy()
            for item in temp:
                if not item.endswith('labelIds.png', 0, len(item)):
                    list_items.remove(item)

            # defining paths
            list_items = ['/'+city+'/'+path for path in list_items]

            self.yLabel_list.extend(list_items)
            self.XImg_list.extend(
                ['/'+city+'/'+path for path in os.listdir(self.rgb_path+'/'+city)]
            )
                
    def __len__(self):
        length = len(self.XImg_list)
        return length      

    def __getitem__(self, index):
        image = Image.open(self.rgb_path+self.XImg_list[index])
        y = Image.open(self.label_path+self.yLabel_list[index])

        if self.transform is not None:
            image = self.transform(image)
            y = self.transform(y)

        image = transforms.ToTensor()(image)
        y = np.array(y)
        y = torch.from_numpy(y)
        y = y.type(torch.LongTensor)
        if self.eval:
            return image, y, self.XImg_list[index]
        else:
            return image, y

def get_cityscapes_data(mode,split,root_dir='datasets\cityscapes',transforms=None,batch_size=1,eval=False,shuffle=True,pin_memory=True):
    data = CityscapesDataset(
        mode=mode, split=split, transform=transforms, root_dir=root_dir, eval=eval)

    data_loaded = torch.utils.data.DataLoader(
        data, batch_size=batch_size, shuffle=shuffle, pin_memory=pin_memory)

    return data_loaded

# Functions to save predictions as images 
def save_as_images(tensor_pred, folder, image_name):
    tensor_pred = transforms.ToPILImage()(tensor_pred.byte())
    filename = f"{folder}\{image_name}.png"
    tensor_pred.save(filename)

In [12]:
class UNET(nn.Module):
    
    def __init__(self, in_channels=3, classes=1):
        super(UNET, self).__init__()
        self.layers = [in_channels, 64, 128, 256]
        self.double_conv_downs = nn.ModuleList(
            [self.__double_conv(layer, layer_n) for layer, layer_n in zip(self.layers[:-1], self.layers[1:])]) 
        self.up_trans = nn.ModuleList(
            [nn.ConvTranspose2d(layer, layer_n, kernel_size=2, stride=2)
             for layer, layer_n in zip(self.layers[::-1][:-2], self.layers[::-1][1:-1])])
        self.double_conv_ups = nn.ModuleList(
        [self.__double_conv(layer, layer//2) for layer in self.layers[::-1][:-2]])
        self.max_pool_2x2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.final_conv = nn.Conv2d(64, classes, kernel_size=1)
    def __double_conv(self, in_channels, out_channels):
        conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True)
        )
        return conv
    def forward(self, x):
        # down layers
        concat_layers = []
        
        for down in self.double_conv_downs:
            x = down(x)
            if down != self.double_conv_downs[-1]:
                concat_layers.append(x)
                x = self.max_pool_2x2(x)
        
        concat_layers = concat_layers[::-1]
        
        # up layers
        for up_trans, double_conv_up, concat_layer  in zip(self.up_trans, self.double_conv_ups, concat_layers):
            x = up_trans(x)
            if x.shape != concat_layer.shape:
                x = TF.resize(x, concat_layer.shape[2:])
            
            concatenated = torch.cat((concat_layer, x), dim=1)
            x = double_conv_up(concatenated)
            
        x = self.final_conv(x)
        
        return x 


In [13]:
if torch.cuda.is_available():
    DEVICE = 'cuda:0'
    print('Running on the GPU')
else:
    DEVICE = "cpu"
    print('Running on the CPU')

Running on the GPU


In [14]:
def accuracy_calculation(pred, label):
    torch.cuda.empty_cache()
    pred_class = torch.argmax(pred, dim=1) 
    pred_class = pred_class.float()
    acc_sum = (pred_class == label).sum()
    acc = float(acc_sum) / torch.numel(label)
    return acc

def acc_epoch(data, model,device):
    acc = []
    for index, batch in enumerate(data): 
        if(index < 20):
            X, y = batch
            X, y = X.to(device), y.to(device)
            preds = model(X)
            acc_ = accuracy_calculation(preds,y)
            acc.append(acc_)
        else:
            break
    return np.mean(acc)


In [35]:
ROOT_DIR = "datasets\\cityscapes"
IMG_HEIGHT = 110  
IMG_WIDTH = 220  
BATCH_SIZE = 32 
LEARNING_RATE = 0.0003
EPOCHS = 50

transform = transforms.Compose([
    transforms.Resize((IMG_HEIGHT, IMG_WIDTH), interpolation=Image.NEAREST),
]) 

train_set = get_cityscapes_data(
    split='train',
    mode='fine',
    root_dir=ROOT_DIR,
    transforms=transform,
    batch_size=BATCH_SIZE,
)

test_set = get_cityscapes_data(
    split='val',
    mode='fine',
    root_dir=ROOT_DIR,
    transforms=transform,
    batch_size=16,
)

C:\Users\ASUS\AppData\Local\Temp\ipykernel_20236\506324850.py:9: DeprecationWarning: NEAREST is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.NEAREST or Dither.NONE instead.
  transforms.Resize((IMG_HEIGHT, IMG_WIDTH), interpolation=Image.NEAREST),


In [16]:
unet = UNET(in_channels=3, classes=35).to(DEVICE).train()
print(acc_epoch(test_set,unet,DEVICE))

0.008260717975206614


In [17]:
def train_function(data, model, optimizer, loss_fn, device):
    print('Entering into train function')
    loss_values = []
    data = tqdm(data)
    for index, batch in enumerate(data): 
        X, y = batch
        X, y = X.to(device), y.to(device)
        preds = model(X)
        loss = loss_fn(preds, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    return loss.item()

LOSS_VALS = []
ACC_VALS = []

unet = UNET(in_channels=3, classes=35).to(DEVICE).train()
optimizer = optim.Adam(unet.parameters(), lr=LEARNING_RATE)
loss_function = nn.CrossEntropyLoss().to(DEVICE)

for e in range(0, EPOCHS):
    torch.cuda.empty_cache()
    print(f'Epoch: {e}')
    loss_val = train_function(train_set, unet, optimizer, loss_function, DEVICE)
    LOSS_VALS.append(loss_val) 
    print(f'Loss = {loss_val}')
    torch.cuda.empty_cache()
    acc = acc_epoch(test_set, unet, DEVICE)
    ACC_VALS.append(acc)
    print(f'Acc = {acc}')

Epoch: 0
Entering into train function


100%|██████████| 93/93 [02:57<00:00,  1.91s/it]


Loss = 1.0940582752227783
Acc = 0.05646126033057851
Epoch: 1
Entering into train function


100%|██████████| 93/93 [02:15<00:00,  1.46s/it]


Loss = 0.893613874912262
Acc = 0.06900490702479338
Epoch: 2
Entering into train function


...

Loss = 0.3065778315067291
Acc = 0.07734581611570249
Epoch: 48
Entering into train function


100%|██████████| 93/93 [02:14<00:00,  1.45s/it]


Loss = 0.23653779923915863
Acc = 0.07732605888429753
Epoch: 49
Entering into train function


100%|██████████| 93/93 [02:14<00:00,  1.45s/it]


Loss = 0.23313070833683014
Acc = 0.07697494834710743


In [37]:
def accuracy_calculation(pred, label):
    torch.cuda.empty_cache()
    pred_class = torch.argmax(pred, dim=1) 
    pred_class = pred_class.float()
    acc_sum = (pred_class == label).sum()
    acc = float(acc_sum) / torch.numel(label)
    return acc

def acc_epoch(data, model,device):
    acc = []
    for index, batch in enumerate(data): 
        if(index < 20):
            X, y = batch
            X, y = X.to(device), y.to(device)
            preds = model(X)
            acc_ = accuracy_calculation(preds,y)
            acc.append(acc_)
        else:
            break
    return np.mean(acc)


c:\Users\ASUS\.conda\envs\pytorch\lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


0.8438272210743802